In [1]:
from itertools import zip_longest
from matplotlib.pyplot import figure
import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from scipy.sparse import csr_array
from scipy.sparse import find
from pickleshare import PickleShareDB

df = pd.read_csv('../input/abcnews-date-text.csv')
nltk.download('stopwords')
stopwords_set = set(stopwords.words('english'))



[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#tokenize and sanitize

#tokenize documents into individual words
df['tokenized'] = df.headline_text.str.split(' ')

#remove short documents from corpus
df['length'] = df.tokenized.map(len)
df = df.loc[df.length > 1]

#use random subset of corpus
df=df.sample(frac=0.0001)

#flatten all words into single series
ex = df.explode('tokenized')

#remove shorter words
ex = ex.loc[ex.tokenized.str.len() > 2]

#remove stopwords
ex = ex.loc[~ex.tokenized.isin(stopwords_set)]

In [3]:
#ceate dictionary of words

#shuffle for sparse matrix visual
dictionary = ex.tokenized.drop_duplicates().sample(frac=1)

#dataframe with (index/code):word
dictionary = pd.Series(dictionary.tolist(), name='words').to_frame()

#store code:word dictionary for reverse encoding
dictionary_lookup = dictionary.to_dict()['words']

#offset index to prevent clash with zero fill
dictionary['encode'] = dictionary.index + 1

#store word:code dictionary for encoding
dictionary = dictionary.set_index('words').to_dict()['encode']


In [4]:
#reduced grouping from group by dataframeto to group by series and aggregated to tuple
#improved from 30s to 20s with whole dataset

#use dictionary to encode each word to integer representation
encode = ex.tokenized.map(dictionary.get).to_frame()
encode.index.astype('int')
encode.tokenized.astype('int')
#un-flatten encoded words back into original documents
docs = encode.tokenized.groupby(level=0).agg(tuple)


In [5]:

#zero pad x dimention by longest sentence
encoded_docs = list(zip(*zip_longest(*docs.to_list(), fillvalue=0)))

#convert to sparse matrix
encoded_docs = csr_array(encoded_docs, dtype=int)

#convert to index for each word
row_column_code = find(encoded_docs)

#presort by words
word_sorted_index = row_column_code[2].argsort()
doc_word = np.array([row_column_code[0][word_sorted_index], row_column_code[2][word_sorted_index]])

#presort by docs and words
doc_word_sorted_index = doc_word[0].argsort()


tfidf = pd.DataFrame(np.array([doc_word[0][doc_word_sorted_index], doc_word[1][doc_word_sorted_index]]).T, columns=['doc','word'])

#offset code no longer needed after zerofill
tfidf.word = tfidf.word - 1


In [6]:
#Term Frequency
#(num_term_in_doc)/(num_words_in_doc)
#Inverse Document Frequency
#log(num_documents/num_docs_with_term)
#Term Frequency Inverse Document Frequency
#(num_term_in_doc)/(num_words_in_doc) * log(num_documents/num_docs_with_term)

tfidf = tfidf.groupby(['doc','word']).size().to_frame('num_term_in_doc').reset_index()

tfidf['num_words_in_doc'] = tfidf.groupby('doc')['num_term_in_doc'].transform('sum')

tfidf['num_documents'] = tfidf.doc.nunique()

tfidf['num_docs_with_term'] = tfidf.groupby('word')['doc'].transform('count')

tfidf

,doc,word,num_term_in_doc,num_words_in_doc,num_documents,num_docs_with_term
0,0,35,1,6,124,3
442,87,464,1,6,124,2
443,88,178,1,6,124,1
444,88,197,1,6,124,1
445,88,209,1,6,124,1
...,...,...,...,...,...,...
225,43,406,1,6,124,1
226,43,446,1,6,124,2
227,43,484,1,6,124,2
229,44,215,1,5,124,1
